In [1]:
import numpy as np
import sqlite3

import torch
from tensordict import TensorDict
import os
from pathlib import Path

In [2]:
heuristic_type = 0
TEST_OUTPUT_DIR = f"/mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/output/{heuristic_type}"
MULTI_BINARY_ACTION_COUNT = 7
print("Current working directory:", os.getcwd())

Current working directory: /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering


In [3]:
TEST_OUTPUT_DIR

'/mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/output/0'

In [4]:
# Get all directories in the test output directory, include basepath
import os
def get_all_directories(basepath):
    directories = []
    for root, dirs, files in os.walk(basepath):
        for dir in dirs:
            directories.append(os.path.join(root, dir))
    return directories

In [5]:
# Using sqlite3 read the database database.db, extract reward from the table grafana
def get_reward_from_db(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    cursor.execute("SELECT reward FROM grafana")
    rows = cursor.fetchall()
    conn.close()
    return np.array(rows)

In [6]:
def sim_to_tensordict(observations, actions, rewards, min_step=98):
    # Assumes replay_buffer is a dict of numpy arrays
    return TensorDict({
        "observation": torch.tensor(observations[:min_step], dtype=torch.float32),
        "action": torch.tensor(actions[:min_step], dtype=torch.long),
        "reward": torch.tensor(rewards[:min_step], dtype=torch.float32),
        "next_observation": torch.tensor(observations[1:min_step+1], dtype=torch.float32),
        "done": torch.tensor(torch.zeros(min_step, dtype=torch.bool), dtype=torch.bool),
    }, batch_size=[min_step])

In [8]:
def process_simulations(sim_path):
    sims = get_all_directories(sim_path)

    for sim in sims:
        try:
            # process observations
            td_dir = Path(os.getcwd())/"raw"/str(heuristic_type)/os.path.basename(sim)
            # check if td_dir exists
            if os.path.exists(td_dir):
                print(f"TensorDict {td_dir} already exists, skipping...")
                continue

            observations = np.genfromtxt(os.path.join(sim, "observation.csv"), delimiter=",")
            observations.shape

            # process actions
            actions = np.genfromtxt(os.path.join(sim, "EsActions.txt"), delimiter=",", skip_header=1)
            assert actions.shape[0]%MULTI_BINARY_ACTION_COUNT == 0, "The number of actions is not divisible by the number of action types"
            actions = actions[:, 2:].reshape(-1, MULTI_BINARY_ACTION_COUNT)

            # process rewards
            rewards = get_reward_from_db(os.path.join(sim, "database.db"))

            min_step = min(actions.shape[0], observations.shape[0], rewards.shape[0])
            min_step = min_step if min_step < observations.shape[0] else min_step - 1

            td = sim_to_tensordict(observations, actions, rewards, min_step=min_step)
            # get the file name without the path
            
            print(f"Saving to {td_dir}, shape: {td.shape}")
            td.memmap_(f"{td_dir}", copy_existing=True)
        except Exception as e:
            print(f"Error processing simulation {sim}: {e}")
            continue

process_simulations(sim_path=TEST_OUTPUT_DIR)

TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/000952ab-e1dd-4885-9729-c419e9c43f20 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/054660c0-e809-4ee5-8a82-fc527f1919d3 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/087da820-512d-423f-9631-952c35b25861 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/08a9dcda-4ede-46dc-a798-8d8a46bf93bb already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/099f501f-ed9c-4ffb-9530-6fcead352df1 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/0a453f4e-ffaa-4c2e-a6e4-672a4f41fb32 already exists, skipping...
TensorDict /mnt/d/Workspace/Networ

/tmp/ipykernel_8984/1246228852.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  "done": torch.tensor(torch.zeros(min_step, dtype=torch.bool), dtype=torch.bool),


Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6a9309ef-673c-4bad-857c-c5d26e5f150a, shape: torch.Size([98])
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6b29c89d-1bcc-47b5-b9fb-c79471640f69, shape: torch.Size([30])
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6bd170c0-5f42-409e-a170-2036f53998a8, shape: torch.Size([98])
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6cedc3ea-164f-450b-9c53-ed9ffa0a3026, shape: torch.Size([98])
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6de61212-b080-4383-b4ef-028b9102bf41 already exists, skipping...
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/6e0a283e-35fc-4609-956a-2833cd8d0d68, shape: torch.Size([36])
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-r

/tmp/ipykernel_8984/3585985119.py:13: UserWarning: genfromtxt: Empty input file: "/mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/output/0/new/9fb545fc-9ad2-4c15-bb4f-235d966d8a6e/observation.csv"
  observations = np.genfromtxt(os.path.join(sim, "observation.csv"), delimiter=",")


Error processing simulation /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/output/0/new/9fb545fc-9ad2-4c15-bb4f-235d966d8a6e: Trying to create tensor with negative dimension -1: [-1]
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/a360424e-7df5-47df-ac47-a7b0c661a819, shape: torch.Size([14])
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/a40c9a9a-4c65-45ba-8487-b8b014ced43d already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/a49673c9-4e51-4b65-a045-a47ff1879e74 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/a5276d40-d9d7-43a5-89b3-ab1e83f449e2 already exists, skipping...
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/a52c9a8e-ec17-47db-9a2a-e650446badf9, shape: torch.Size([98])
TensorDict /mnt/d

/tmp/ipykernel_8984/3585985119.py:13: UserWarning: genfromtxt: Empty input file: "/mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/output/0/new/dfbc7332-b461-4560-a9ce-d6ebf08e9098/observation.csv"
  observations = np.genfromtxt(os.path.join(sim, "observation.csv"), delimiter=",")


Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e057133c-177c-447b-b9d1-fd744c3dab30, shape: torch.Size([98])
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e1656ec1-2c85-4b89-8d35-e8920a7c5538, shape: torch.Size([79])
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e341389a-75a3-443c-b41b-21700d0e4e78 already exists, skipping...
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e3ebac09-5b21-44ff-beeb-56365d493da7 already exists, skipping...
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e56b10bf-d02c-4ba6-8259-b75a3e08d0a2, shape: torch.Size([98])
Saving to /mnt/d/Workspace/Networks/RL-Project/ns-o-ran-gym/dev/explore/data-engineering/raw/0/e5c4e1c8-8c64-406c-8968-da14d876f0f4, shape: torch.Size([98])
TensorDict /mnt/d/Workspace/Networks/RL-Project/ns